In [1]:
import pandas as pd
import requests
!pip install beautifulsoup4

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011


In [2]:
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
page.text
soup = BeautifulSoup(page.text,'html.parser')

In [4]:
#tab=soup.find("table",{"class":"wikitable sortable"})
#print(tab)
table=soup.find_all("table",class_="wikitable sortable")
len(table)

1

In [5]:
table = soup.find_all('table')[0] 
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])
print(df)

    PostalCode           Borough           Neighborhood
0          M1A      Not assigned           Not assigned
1          M2A      Not assigned           Not assigned
2          M3A        North York              Parkwoods
3          M4A        North York       Victoria Village
4          M5A  Downtown Toronto           Harbourfront
..         ...               ...                    ...
283        M8Z         Etobicoke              Mimico NW
284        M8Z         Etobicoke     The Queensway West
285        M8Z         Etobicoke  Royal York South West
286        M8Z         Etobicoke         South of Bloor
287        M9Z      Not assigned           Not assigned

[288 rows x 3 columns]


In [6]:
df2=df[df.Borough != 'Not assigned']

In [7]:
df2.reset_index(drop=True)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West
207,M8Z,Etobicoke,Mimico NW
208,M8Z,Etobicoke,The Queensway West
209,M8Z,Etobicoke,Royal York South West


In [8]:
table2 = df2.groupby(by=['PostalCode','Borough']).agg(lambda x: ','.join(x))
table2.reset_index(level=['PostalCode','Borough'], inplace=True)
table2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [9]:
table2['Neighborhood'].loc[table2.Neighborhood=="Not assigned"]=table2['Borough']

In [10]:
table2.loc[table2.Neighborhood=="Queen's Park"]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [11]:
table2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [13]:
table2.shape

(103, 3)

In [14]:
import numpy as np

In [15]:
!conda install -c anaconda xlrd --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    xlrd-1.2.0                 |           py36_0         188 KB  anaconda
    certifi-2019.6.16          |           py36_1         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1  conda-forge --> 2019.6.16-py36_1 anaconda
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1-h7b6

In [19]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')
print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [20]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
# Merge the dataframes into another dataframe based on PERSONID and Badge_ID
table3 = pd.merge(table2, geo_data, how='outer', on=table2['PostalCode'])

In [24]:
table3 = table3.drop("Postal Code", axis=1)

In [25]:
table3

,key_0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,M9N,York,Weston,43.706876,-79.518188
99,M9P,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


In [26]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(table3['Borough'].unique()),
        table3.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.
